In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
project_path = os.getenv("PROJECT_PATH")

In [2]:
%run "{project_path}\llm_custom_apps\llm_apps\oh_sheet_its_spark\utils.ipynb"

13


In [3]:
xls_col_info = get_excel_col_header(f"{test_data_path}\SalesData1.xlsx")
xls_col_info

{'SalesInfo': {'StoreID': {'ColumnID': 'A'},
  'SalesYear': {'ColumnID': 'B'},
  'SalesValue': {'ColumnID': 'C'},
  'StoreLocation': {'ColumnID': 'D'},
  'Owner': {'ColumnID': 'E'}},
 'RegionInfo': {'ID': {'ColumnID': 'A'}, 'Location': {'ColumnID': 'B'}},
 'StoreInfo': {'ID': {'ColumnID': 'A'},
  'RegionID': {'ColumnID': 'B'},
  'StoreOwner': {'ColumnID': 'C'}},
 'SaleGrowthReport1': {'StoreID': {'ColumnID': 'A'},
  'AverageSales': {'ColumnID': 'B'},
  'MinSales': {'ColumnID': 'C'},
  'MaxSales': {'ColumnID': 'D'}},
 'SaleGrowthReport2': {'StoreLocation-SalesYear': {'ColumnID': 'A'},
  'Year': {'ColumnID': 'B'},
  'Location': {'ColumnID': 'C'},
  'AverageSales': {'ColumnID': 'D'},
  'MinSales': {'ColumnID': 'E'},
  'MaxSales': {'ColumnID': 'F'}}}

In [4]:
xls_str = process_excel_columns_xlwings(f"{test_data_path}\SalesData1.xlsx",xls_col_info)
xls_str

{'SalesInfo': {'StoreID': {'ColumnID': 'A', 'ColumnValue': '11'},
  'SalesYear': {'ColumnID': 'B', 'ColumnValue': '2020'},
  'SalesValue': {'ColumnID': 'C', 'ColumnValue': '15789'},
  'StoreLocation': {'ColumnID': 'D',
   'ColumnValue': '=IFERROR(INDEX(RegionInfo!B:B, MATCH(INDEX(StoreInfo!B:B, MATCH(A2, StoreInfo!A:A, 0)), RegionInfo!A:A, 0)), "")'},
  'Owner': {'ColumnID': 'E',
   'ColumnValue': '=IFERROR(INDEX(StoreInfo!C:C, MATCH(A2, StoreInfo!A:A, 0)), "")'}},
 'RegionInfo': {'ID': {'ColumnID': 'A', 'ColumnValue': '1'},
  'Location': {'ColumnID': 'B', 'ColumnValue': 'Halifax'}},
 'StoreInfo': {'ID': {'ColumnID': 'A', 'ColumnValue': '11'},
  'RegionID': {'ColumnID': 'B', 'ColumnValue': '1'},
  'StoreOwner': {'ColumnID': 'C', 'ColumnValue': 'Ethan Carter'}},
 'SaleGrowthReport1': {'StoreID': {'ColumnID': 'A',
   'ColumnValue': '=_xlfn.UNIQUE(_xlfn._xlws.FILTER(SalesInfo!A:A, SalesInfo!A:A<>""))'},
  'AverageSales': {'ColumnID': 'B',
   'ColumnValue': '=AVERAGEIF(SalesInfo!A:A, A2, S

In [5]:
messages = [
    {"role": "system", "content": sheet_spark_prompt},
    {"role": "user", "content": f"Convert the excel logic representation to equivalent pyspark code : {xls_str}"},
  ]

In [6]:
generate_responses("JasjyotSinghJaswal/code_qwen_1.5_7b_quant",messages)

Loading model: JasjyotSinghJaswal/code_qwen_1.5_7b_quant...


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Error occurred: Failed to import transformers.integrations.bitsandbytes because of the following error (look up to see its traceback):
[WinError 193] %1 is not a valid Win32 application
CUDA cache cleared
